## EDA - Task 3

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os

In [5]:
import datetime as dt

In [2]:
flights = pd.read_csv('data/flights_10000.csv', index_col=0)
flights.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-07-10,WN,WN,WN,2212,WN,N958WN,2212,14683,SAT,...,822.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-10-10,UA,UA,UA,206,UA,N68811,206,13930,ORD,...,1005.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-07-18,AA,AA_CODESHARE,AA,4221,MQ,N694AE,4221,12891,LAW,...,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-11-25,UA,UA_CODESHARE,UA,4822,ZW,NaN,4822,14685,SAV,...,773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-10-24,DL,DL,DL,957,DL,N693DL,957,11433,DTW,...,1960.0,0.0,0.0,28.0,0.0,0.0,NaN,NaN,NaN,NaN


In [3]:
flights.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

## **Task 3**: Does the weather affect the delay? 
Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:
- sunny
- cloudy
- rainy
- snow.
Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?

In [307]:
#select the relevant cols 
weather_df = flights[['fl_date', 'origin_city_name','crs_dep_time', 'dest_city_name', 'crs_arr_time']].head(10)
weather_df

,fl_date,origin_city_name,crs_dep_time,dest_city_name,crs_arr_time
0,2018-07-10,"San Antonio, TX",705,"Nashville, TN",915
1,2019-10-10,"Chicago, IL",800,"Orlando, FL",1148
2,2019-07-18,"Lawton/Fort Sill, OK",1034,"Dallas/Fort Worth, TX",1130
3,2018-11-25,"Savannah, GA",2200,"Chicago, IL",2327
4,2018-10-24,"Detroit, MI",841,"Santa Ana, CA",1039
5,2019-06-08,"Richmond, VA",1149,"Dallas/Fort Worth, TX",1400
6,2018-01-29,"Charlotte, NC",1436,"St. Louis, MO",1534
7,2019-08-14,"Allentown/Bethlehem/Easton, PA",1844,"Sanford, FL",2105
8,2019-01-28,"Tucson, AZ",735,"Dallas/Fort Worth, TX",1050
9,2018-03-05,"Charlotte, NC",1759,"Raleigh/Durham, NC",1854


In [235]:
import requests
import time

In [220]:
#let's create a list of all city/date combinations to see if we can be more efficient with our api pulls
city_date = flights[['fl_date', 'origin_city_name']].rename({'origin_city_name': 'city'}, axis = 1)    #start with origin cities
temp = flights[['fl_date', 'dest_city_name']].rename({'dest_city_name': 'city'}, axis = 1)             #dest cities

In [226]:
#append dest cities to origin cities
city_date = pd.concat([city_date, temp], ignore_index=True)

In [229]:
city_date.shape        #confirm the df doubles in size

(20000, 2)

In [231]:
#find unique city_date combos
city_date.groupby(['fl_date','city']).size().reset_index().rename(columns={0:'count'}).sort_values(by = 'count', ascending = False)

,fl_date,city,count
6240,2018-10-25,"New York, NY",7
11766,2019-07-10,"Chicago, IL",7
13671,2019-10-02,"Chicago, IL",7
11589,2019-07-01,"Chicago, IL",7
14817,2019-11-23,"Denver, CO",7
...,...,...,...
5819,2018-10-05,"Baltimore, MD",1
5820,2018-10-05,"Boston, MA",1
5821,2018-10-05,"Charlotte, NC",1
5826,2018-10-05,"Fort Lauderdale, FL",1


unique combos only reduce the total pulls by 4370. In the grand scheme of things this is not enough of a reducion to go this route.

In [324]:
#open weather api function
def open_weather_api(lat, long, date, timezone = "America%2FChicago"):
    """
    function returns individual api calls by location and date. 
    """
    #timezone = America%2FChicago #timezone format, opportunity to fine tune this - doesn't work for some reason, I think formatting
    
    url = f"https://archive-api.open-meteo.com/v1/era5?latitude={lat}&longitude={long}&start_date={date}&end_date={date}&hourly=precipitation,rain,snowfall,cloudcover&daily=precipitation_sum,rain_sum,snowfall_sum,precipitation_hours&timezone={timezone}"
    response = requests.get(url)
    return response.json()

In [321]:
test_api = open_weather_api("41.75","-87.75","2018-10-25", "America%2FNew_York")

In [322]:
test_api

{'latitude': 41.75,
 'longitude': -87.75,
 'generationtime_ms': 0.3420114517211914,
 'utc_offset_seconds': -14400,
 'timezone': 'America/New_York',
 'timezone_abbreviation': 'EDT',
 'elevation': 192.0,
 'hourly_units': {'time': 'iso8601',
  'precipitation': 'mm',
  'rain': 'mm',
  'snowfall': 'cm',
  'cloudcover': '%'},
 'hourly': {'time': ['2018-10-25T00:00',
   '2018-10-25T01:00',
   '2018-10-25T02:00',
   '2018-10-25T03:00',
   '2018-10-25T04:00',
   '2018-10-25T05:00',
   '2018-10-25T06:00',
   '2018-10-25T07:00',
   '2018-10-25T08:00',
   '2018-10-25T09:00',
   '2018-10-25T10:00',
   '2018-10-25T11:00',
   '2018-10-25T12:00',
   '2018-10-25T13:00',
   '2018-10-25T14:00',
   '2018-10-25T15:00',
   '2018-10-25T16:00',
   '2018-10-25T17:00',
   '2018-10-25T18:00',
   '2018-10-25T19:00',
   '2018-10-25T20:00',
   '2018-10-25T21:00',
   '2018-10-25T22:00',
   '2018-10-25T23:00'],
  'precipitation': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
  

In [256]:
#api requires lat/long, found a csv of american cities and lat/longs
lat_long_df = pd.read_csv('data/uscities.csv')
lat_long_df = lat_long_df[['city', 'state_id', 'lat', 'lng', 'timezone']]

In [262]:
#create a column, city_state, to match our main city column
lat_long_df['city_state'] = lat_long_df['city'] + ", " + lat_long_df['state_id']

In [263]:
lat_long_df.head()

,city,state_id,lat,lng,timezone,city_state
0,New York,NY,40.6943,-73.9249,America/New_York,"New York, NY"
1,Los Angeles,CA,34.1141,-118.4068,America/Los_Angeles,"Los Angeles, CA"
2,Chicago,IL,41.8375,-87.6866,America/Chicago,"Chicago, IL"
3,Miami,FL,25.7840,-80.2101,America/New_York,"Miami, FL"
4,Dallas,TX,32.7935,-96.7667,America/Chicago,"Dallas, TX"


In [318]:
#attempt to fix the timezone formatting
lat_long_df['timezone'] = lat_long_df['timezone'].str.replace("/", "%2F")

In [319]:
lat_long_df

,city,state_id,lat,lng,timezone
city_state,,,,,
"New York, NY",New York,NY,40.6943,-73.9249,America%2FNew_York
"Los Angeles, CA",Los Angeles,CA,34.1141,-118.4068,America%2FLos_Angeles
"Chicago, IL",Chicago,IL,41.8375,-87.6866,America%2FChicago
"Miami, FL",Miami,FL,25.7840,-80.2101,America%2FNew_York
"Dallas, TX",Dallas,TX,32.7935,-96.7667,America%2FChicago
...,...,...,...,...,...
"Drummond, ID",Drummond,ID,43.9996,-111.3433,America%2FBoise
"Lost Springs, WY",Lost Springs,WY,42.7652,-104.9255,America%2FDenver
"Provo, SD",Provo,SD,43.1937,-103.8329,America%2FDenver


In [329]:
#create empty cols in weather df to be filled with the column filler function
weather_df['weather_origin'] = 0
weather_df['weather_dest'] = 0
weather_df.head()

,fl_date,origin_city_name,crs_dep_time,dest_city_name,crs_arr_time,weather_origin,weather_dest
0,2018-07-10,"San Antonio, TX",705,"Nashville, TN",915,0,0
1,2019-10-10,"Chicago, IL",800,"Orlando, FL",1148,0,0
2,2019-07-18,"Lawton/Fort Sill, OK",1034,"Dallas/Fort Worth, TX",1130,0,0
3,2018-11-25,"Savannah, GA",2200,"Chicago, IL",2327,0,0
4,2018-10-24,"Detroit, MI",841,"Santa Ana, CA",1039,0,0


In [ ]:
#get rid of the second city name i.e. Lawton/Fort Sill, OK -------> Lawton, OK

In [378]:
#cleaning the city names
state = weather_df['origin_city_name'].str.split(',').apply(lambda x: x[1])       #split city(s) and state
temp = weather_df['origin_city_name'].str.split(',').apply(lambda x: x[0])        #split the multiple states into a list
city_task3 = temp.str.split('/').apply(lambda x: x[0])                            #[0] will be the first city in your city list (temp)

result3 = city_task3 + ',' + state

In [379]:
result3

0    San Antonio, TX
1        Chicago, IL
2         Lawton, OK
3       Savannah, GA
4        Detroit, MI
5       Richmond, VA
6      Charlotte, NC
7      Allentown, PA
8         Tucson, AZ
9      Charlotte, NC
Name: origin_city_name, dtype: object

In [380]:
weather_df['origin_city_name'] = result3

In [ ]:
#replace the above with a func

In [383]:
def city_scrubber(df, target_column):
    """
    update target_column in dataframe df to remove multiple city names with alternate names within string.
    doesn't return anything, but your df will be scrubbed
    """
    state = df[target_column].str.split(',').apply(lambda x: x[1])       #split city(s) and state
    temp = df[target_column].str.split(',').apply(lambda x: x[0])        #split the multiple states into a list
    city = temp.str.split('/').apply(lambda x: x[0])                            #[0] will be the first city in your city list (temp)
    
    result = city + ',' + state
    df[target_column] = result

In [384]:
city_scrubber(weather_df, 'dest_city_name')
#city_scrubber(weather_df, 'origin_city_name')

In [385]:
weather_df

,fl_date,origin_city_name,crs_dep_time,dest_city_name,crs_arr_time,weather_origin,weather_dest
0,2018-07-10,"San Antonio, TX",705,"Nashville, TN",915,sunny,sunny
1,2019-10-10,"Chicago, IL",800,"Orlando, FL",1148,rain,rain
2,2019-07-18,"Lawton, OK",1034,"Dallas, TX",1130,0,0
3,2018-11-25,"Savannah, GA",2200,"Chicago, IL",2327,0,0
4,2018-10-24,"Detroit, MI",841,"Santa Ana, CA",1039,0,0
5,2019-06-08,"Richmond, VA",1149,"Dallas, TX",1400,0,0
6,2018-01-29,"Charlotte, NC",1436,"St. Louis, MO",1534,0,0
7,2019-08-14,"Allentown, PA",1844,"Sanford, FL",2105,0,0
8,2019-01-28,"Tucson, AZ",735,"Dallas, TX",1050,0,0
9,2018-03-05,"Charlotte, NC",1759,"Raleigh, NC",1854,0,0


In [338]:
def weather_cols_openweather(df, lat_long_df = lat_long_df):
    """
    Params: 
    df = weather_df
    lat_long_df
    """
    
    locations = ["origin", "dest"]
    
    for index in df.index:
        
        for loc in locations:
            city = df[f'{loc}_city_name'][index]
            date = df['fl_date'][index]
            
            #print(lat_long_df['lat'][lat_long_df.index.str.match(city)])
                                     
            lat = float(lat_long_df['lat'][lat_long_df.index.str.match(city)])
            long = float(lat_long_df['lng'][lat_long_df.index.str.match(city)])
            #timezone = lat_long_df['timezone'][lat_long_df.index.str.match(city)]

            json_result = open_weather_api(lat, long, date)      #remove timezone, not working 
            #print(json_result)

            total_rain = int(json_result['daily']['rain_sum'][0])                  # mm
            total_snow = int(json_result['daily']['snowfall_sum'][0])              # cm
            cloudcover_mean = round(sum(json_result['hourly']['cloudcover'])/len(json_result['hourly']['cloudcover']))     # %

            if total_rain == 0 & total_snow == 0 & cloudcover_mean < 40:
                df[f'weather_{loc}'][index] = "sunny"
            elif total_rain == 0 & total_snow == 0 & cloudcover_mean >= 40:
                df[f'weather_{loc}'][index] = "cloudy"
            elif total_rain > 0 & total_snow == 0:
                df[f'weather_{loc}'][index] = "rain"
            elif total_rain == 0 & total_snow > 0:
                df[f'weather_{loc}'][index] = "snow"
            elif total_rain > 0 & total_snow > 0:
                df[f'weather_{loc}'][index] = "snow & rain"
            else:
                df[f'weather_{loc}'][index] = "error"

In [386]:
weather_cols_openweather(weather_df)

city_state
San Antonio, TX    29.4632
Name: lat, dtype: float64
{'latitude': 29.5, 'longitude': -98.5, 'generationtime_ms': 0.34999847412109375, 'utc_offset_seconds': -18000, 'timezone': 'America/Chicago', 'timezone_abbreviation': 'CDT', 'elevation': 256.0, 'hourly_units': {'time': 'iso8601', 'precipitation': 'mm', 'rain': 'mm', 'snowfall': 'cm', 'cloudcover': '%'}, 'hourly': {'time': ['2018-07-10T00:00', '2018-07-10T01:00', '2018-07-10T02:00', '2018-07-10T03:00', '2018-07-10T04:00', '2018-07-10T05:00', '2018-07-10T06:00', '2018-07-10T07:00', '2018-07-10T08:00', '2018-07-10T09:00', '2018-07-10T10:00', '2018-07-10T11:00', '2018-07-10T12:00', '2018-07-10T13:00', '2018-07-10T14:00', '2018-07-10T15:00', '2018-07-10T16:00', '2018-07-10T17:00', '2018-07-10T18:00', '2018-07-10T19:00', '2018-07-10T20:00', '2018-07-10T21:00', '2018-07-10T22:00', '2018-07-10T23:00'], 'precipitation': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

C:\Users\3_hal\AppData\Local\Temp\ipykernel_15016\2179816563.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'weather_{loc}'][index] = "sunny"


{'latitude': 36.25, 'longitude': -86.75, 'generationtime_ms': 0.34105777740478516, 'utc_offset_seconds': -18000, 'timezone': 'America/Chicago', 'timezone_abbreviation': 'CDT', 'elevation': 184.0, 'hourly_units': {'time': 'iso8601', 'precipitation': 'mm', 'rain': 'mm', 'snowfall': 'cm', 'cloudcover': '%'}, 'hourly': {'time': ['2018-07-10T00:00', '2018-07-10T01:00', '2018-07-10T02:00', '2018-07-10T03:00', '2018-07-10T04:00', '2018-07-10T05:00', '2018-07-10T06:00', '2018-07-10T07:00', '2018-07-10T08:00', '2018-07-10T09:00', '2018-07-10T10:00', '2018-07-10T11:00', '2018-07-10T12:00', '2018-07-10T13:00', '2018-07-10T14:00', '2018-07-10T15:00', '2018-07-10T16:00', '2018-07-10T17:00', '2018-07-10T18:00', '2018-07-10T19:00', '2018-07-10T20:00', '2018-07-10T21:00', '2018-07-10T22:00', '2018-07-10T23:00'], 'precipitation': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'rain': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

C:\Users\3_hal\AppData\Local\Temp\ipykernel_15016\2179816563.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'weather_{loc}'][index] = "rain"


{'latitude': 28.5, 'longitude': -81.25, 'generationtime_ms': 0.3540515899658203, 'utc_offset_seconds': -18000, 'timezone': 'America/Chicago', 'timezone_abbreviation': 'CDT', 'elevation': 23.0, 'hourly_units': {'time': 'iso8601', 'precipitation': 'mm', 'rain': 'mm', 'snowfall': 'cm', 'cloudcover': '%'}, 'hourly': {'time': ['2019-10-10T00:00', '2019-10-10T01:00', '2019-10-10T02:00', '2019-10-10T03:00', '2019-10-10T04:00', '2019-10-10T05:00', '2019-10-10T06:00', '2019-10-10T07:00', '2019-10-10T08:00', '2019-10-10T09:00', '2019-10-10T10:00', '2019-10-10T11:00', '2019-10-10T12:00', '2019-10-10T13:00', '2019-10-10T14:00', '2019-10-10T15:00', '2019-10-10T16:00', '2019-10-10T17:00', '2019-10-10T18:00', '2019-10-10T19:00', '2019-10-10T20:00', '2019-10-10T21:00', '2019-10-10T22:00', '2019-10-10T23:00'], 'precipitation': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.6, 0.0, 0.6, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'rain': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [387]:
weather_df

,fl_date,origin_city_name,crs_dep_time,dest_city_name,crs_arr_time,weather_origin,weather_dest
0,2018-07-10,"San Antonio, TX",705,"Nashville, TN",915,sunny,sunny
1,2019-10-10,"Chicago, IL",800,"Orlando, FL",1148,rain,rain
2,2019-07-18,"Lawton, OK",1034,"Dallas, TX",1130,sunny,sunny
3,2018-11-25,"Savannah, GA",2200,"Chicago, IL",2327,sunny,rain
4,2018-10-24,"Detroit, MI",841,"Santa Ana, CA",1039,sunny,sunny
5,2019-06-08,"Richmond, VA",1149,"Dallas, TX",1400,rain,sunny
6,2018-01-29,"Charlotte, NC",1436,"St. Louis, MO",1534,rain,sunny
7,2019-08-14,"Allentown, PA",1844,"Sanford, FL",2105,rain,rain
8,2019-01-28,"Tucson, AZ",735,"Dallas, TX",1050,sunny,sunny
9,2018-03-05,"Charlotte, NC",1759,"Raleigh, NC",1854,sunny,sunny


### Let's try it out on the flights_test

In [4]:
#load test data (these are the cols we will have at the test phase)
fl_test = pd.read_csv('data/flights_test_10.csv')
fl_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,1577836800000,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,1577836800000,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,1577836800000,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,1577836800000,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,1577836800000,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


In [5]:
pd.Timestamp.fromtimestamp(1577836800000/1000)

Timestamp('2020-01-01 00:00:00')

In [54]:
fl_test['fl_date'] = fl_test['fl_date']/1000
fl_test['fl_date'] = fl_test['fl_date'].apply(pd.Timestamp.fromtimestamp)

In [56]:
fl_test

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333
5,2020-01-01,WN,WN,WN,5684,WN,N7856A,5684,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",600,715,N,75,1,333
6,2020-01-01,WN,WN,WN,6152,WN,N7735A,6152,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1620,1740,N,80,1,333
7,2020-01-01,WN,WN,WN,1679,WN,N405WN,1679,13891,ONT,"Ontario, CA",14893,SMF,"Sacramento, CA",1505,1630,N,85,1,390
8,2020-01-01,WN,WN,WN,3479,WN,N489WN,3479,13891,ONT,"Ontario, CA",14893,SMF,"Sacramento, CA",1230,1355,N,85,1,390
9,2020-01-01,WN,WN,WN,4069,WN,N7708E,4069,13891,ONT,"Ontario, CA",14893,SMF,"Sacramento, CA",740,900,N,80,1,390


In [8]:
fl_test.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance'],
      dtype='object')